In [1]:
!pip install --quiet transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.1 MB/s eta 0:00:00


In [2]:
import json
import tensorflow as tf
import tensorflow.keras.layers as kl
import keras
from keras.layers import LSTM, Embedding, Dense
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
import numpy as np
import zipfile
from datasets import load_dataset





In [4]:
with zipfile.ZipFile("sample_data/corpusCasM2-main.zip", "r") as archive:
        archive.extractall("data")
corpusCas = load_dataset("data/corpusCasM2-main/corpusCasM2")



Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset corpus_cas_m2 downloaded and prepared to /root/.cache/huggingface/datasets/corpus_cas_m2/default/1.0.0/1e18a23d323119b28ca96ca83baeaf676e58908f30122e75a54225833f284766. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
X_train = [i['tokens'] for i in corpusCas["train"]]
Y_train = [i["ner_tags"] for i in corpusCas["train"]]
X_test = [i['tokens'] for i in corpusCas["test"]]
Y_test = [i["ner_tags"] for i in corpusCas["test"]]
X_validation = [i['tokens'] for i in corpusCas["validation"]]
Y_validation = [i["ner_tags"] for i in corpusCas["validation"]]
    

In [6]:
# Initialisation d'un set pour stocker les mots uniques
unique_words = set()

# Parcours de chaque dictionnaire de la liste
for example in X_train:
  # Vérification que l'élément est une liste de mots
  if isinstance(example, list):
    # Ajout des mots uniques au set
    unique_words.update(example)

# Calcul du nombre de mots uniques
num_unique_words = len(unique_words)
print(num_unique_words)

14678


In [7]:
# Créer le tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_unique_words)

In [8]:
# Entrainer le tokenizer sur le train set 

tokenizer.fit_on_texts(X_train)

In [9]:
# Transformer les textes en vecteurs numeriques à l'aide du tokenizer

X_train_seq = tokenizer.texts_to_sequences(X_train)

X_test_seq = tokenizer.texts_to_sequences(X_test)

X_validation_seq = tokenizer.texts_to_sequences(X_validation)

In [10]:
# Padding des sequences 

max_len = 60

X_train_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen = max_len, 
                                                               truncating = "post")
Y_train_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(Y_train, maxlen = max_len, 
                                                               truncating = "post",value=12)

X_test_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen = max_len, 
                                                               truncating = "post")
Y_test_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(Y_test, maxlen = max_len, 
                                                               truncating = "post",value=12)
X_validation_seq_pad  = tf.keras.preprocessing.sequence.pad_sequences(X_validation_seq,
                                                             maxlen = max_len, 
                                                             truncating='post')
Y_validation_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(Y_validation, maxlen = max_len, 
                                                               truncating = "post",value=12)

In [11]:
Y_train_seq_pad_cat = tf.keras.utils.to_categorical(Y_train_seq_pad)

Y_test_seq_pad_cat = tf.keras.utils.to_categorical(Y_test_seq_pad)

Y_validation_seq_pad_cat = tf.keras.utils.to_categorical(Y_validation_seq_pad)

In [12]:
# On définit le modèle
model = Sequential()

# On ajoute une couche d'embedding avec une dimension de 14678
model.add(Embedding(input_dim=num_unique_words, output_dim=100, input_length=60))

# On ajoute une couche LSTM avec 32 neurones
model.add(LSTM(32, return_sequences=True))



# On ajoute une couche de sortie fully-connected avec autant de sorties que d'annotations différentes
output_layer = Dense(13, activation='softmax')
model.add(output_layer)



In [13]:
# On compile le modèle en spécifiant une fonction de perte et un optimiseur
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=[precision, recall])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           1467800   
                                                                 
 lstm (LSTM)                 (None, 60, 32)            17024     
                                                                 
 dense (Dense)               (None, 60, 13)            429       
                                                                 
Total params: 1,485,253
Trainable params: 1,485,253
Non-trainable params: 0
_________________________________________________________________


In [14]:
# On entraîne le modèle en utilisant nos données d'entraînement
model.fit(X_train_seq_pad, Y_train_seq_pad_cat, epochs=5, batch_size=128, validation_data=[X_validation_seq_pad,Y_validation_seq_pad_cat])



Epoch 1/5
65/65 [==============================] - 8s 20ms/step - loss: 1.0735 - precision: 0.9113 - recall: 0.5720 - val_loss: 0.5345 - val_precision: 0.9104 - val_recall: 0.8791
Epoch 2/5
65/65 [==============================] - 1s 10ms/step - loss: 0.4472 - precision: 0.9148 - recall: 0.8868 - val_loss: 0.4134 - val_precision: 0.9195 - val_recall: 0.8924
Epoch 3/5
65/65 [==============================] - 1s 10ms/step - loss: 0.3487 - precision: 0.9388 - recall: 0.8920 - val_loss: 0.3809 - val_precision: 0.9414 - val_recall: 0.8950
Epoch 4/5
65/65 [==============================] - 1s 11ms/step - loss: 0.3051 - precision: 0.9620 - recall: 0.9026 - val_loss: 0.3777 - val_precision: 0.9453 - val_recall: 0.8922
Epoch 5/5
65/65 [==============================] - 1s 11ms/step - loss: 0.2802 - precision: 0.9705 - recall: 0.9013 - val_loss: 0.3733 - val_precision: 0.9505 - val_recall: 0.8867


In [15]:
# On évalue le modèle en utilisant nos données de test
scores = model.evaluate(X_test_seq_pad, Y_test_seq_pad_cat)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print( "\n%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))

preci = scores[1]*100
rec = scores[2]*100
print("F1_score = %.2f%%" % (2 * (preci * rec) / (preci + rec)))

80/80 [==============================] - 0s 5ms/step - loss: 0.3595 - precision: 0.9530 - recall: 0.8884

precision: 95.30%

recall: 88.84%
F1_score = 91.96%
